In [1]:
from pathlib import Path
import sys
#sys.path.append('/home/elgutto/Documents/masters/goexplore/atari/lib/python3.8/site-packages')
import gym
import json
from collections import namedtuple

In [46]:
demo

i = len(self) - 1 = 2244
len(self): 2245


Demonstration(score: 21.0, timesteps: 2245, env: PongDeterministic-v4)

In [47]:
class Demonstration:
    def __init__(self, experiences, env):
        self.experiences = experiences
        self.env = env
        self.starting_point = len(self) - 1
        
    def restore(self, i):
        state = self.experiences[i].state
        self.env.unwrapped.restore_state(state) # env as parameter?
        return state
    
    def reward(self, i):
        return self.experiences[i].reward
    
    def score(self, i):
        return self.experiences[i].score
    
    def action(self, i):
        return self.experiences[i].action
    
    def is_lagging(self, score, i, L):
        window = self.experiences[i - L, i + L + 1]
        window_scores = [experience.score for experience in window]
        return score < min(window_scores)
    
    def update_starting_point(self, starting_point):
        self.starting_point = starting_point
    
    def __len__(self):
        return len(self.experiences)
    
    def __eq__(self, other):
        return len(self) == len(other)
    
    def __lt__(self, other):
        return len(self) < len(other)
    
    def __repr__(self):
        return (f'Demonstration(score: {self.score(len(self)-1)}, '
                              f'timesteps: {len(self)}, '
                              f'env: {self.env.unwrapped.spec.id})')
    
def read_actions(json_file):
    with open(json_file) as f:
        d = json.load(f)
        return d['actions_taken']
    
Experience = namedtuple('Experience', 'state action reward done score')
def demo_from_actions(actions, env):
    env.reset()
    done = False
    experiences = []
    score = 0
    for action in actions:
        state, reward, done, _ = env.step(action)
        score += reward
        experiences.append(Experience(state, action, reward, done, score))
        if done: break
    demonstration = Demonstration(experiences, env)
    return demonstration

p = Path('explore/experiments/2021-12-13-13:29:34_demos4M').glob('*')
files = [x for x in p if x.is_file()]
demonstrations = []
for file in files:
    env = gym.make('PongDeterministic-v4')
    actions = read_actions(file)
    demonstrations.append(demo_from_actions(actions, env))

In [48]:
demo = sorted(demonstrations)[0]

In [49]:
demo

Demonstration(score: 21.0, timesteps: 2245, env: PongDeterministic-v4)

In [14]:
sp = 870
D = 1
import numpy as np
np.random.choice([sp - i for i in range(D)])

870

In [ ]:
# Single demo

K = 800
L = 128 # Batch size I guess?
delta = 100
env = gym.make('PongDeterministic-v4')
starting_point = demo.starting_point
D = 1
candidate_start_points = [starting_point - i for i in range(D)]
tau = np.random.choice(candidate_starting_points)
score = demo.score(tau)
state = demo.restore(tau)
i = tau - K

while True: #for _ in range(1000000)
    policy = optimizer.policy()
    starting_point = optimizer.starting_point()
    W = False
    D = []
    
    for _ in range(L): #?
        if i >= tau:
            action = policy(state)
            mask = True
            next_state, reward, done, _ = env.step(action)
            score += reward
        else:
            state = demo.restore(i)
            action = demo.action(i)
            reward = demo.reward(i)
            next_state = demo.state(i + 1)
            done = demo.done(i)
            mask = False
        
        D.append((state, action, reward, next_state, done, mask))
        i += 1
        
        if done:
            #W = True if score >= demo.score(i) # Push back as soon as it completes
            if score >= demo.score(i):
                demo.update_starting_point(demo.starting_point - delta)
            starting_point = demo.starting_point
            candidate_start_points = [starting_point - i for i in range(D)]
            tau = np.random.choice(candidate_starting_points)
            score = demo.score(tau)
            i = tau - K
            demo.restore(tau)
    optimizer.optimize(D)

In [ ]:
env = gym.make('PongDeterministic-v4')
S_demo = 3 # Current starting state index for demo; zero or none I think, start_frame or starting_points
L = 50 # Allowed lag
K = 800 # Steps to initialize RNN state K (800? ffmemsize?)
C = 7 # Extra frames coefficient 
policy = lambda state: np.random.randint(3)

In [5]:
frame_counts_demo = []
frame_counts_virtual = []
done = True
while true:
    if done:
        w_demo = len(demonstrations) / np.mean(frame_counts_demo) if frame_counts_demo else 1
        w_virtual = 1 / np.mean(frame_counts_virtual) if frame_counts_virtual else 1
        p_virtual = w_virtual / (w_demo + w_virtual)
        
        if np.random.rand() < p_virtual:
            demo = None
            i = 0
            state = env.reset()
        else:
            demo = np.random.choice(demonstrations)
            i = max(0, demo.starting_point - K)
            state = demo.restore(i)
            score = demo.score(i)
            remaining_frame_count = len(demo) - i + np.exp(C * np.random.rand())
            
    if demo is None or i >= demo.starting_point: # Act according to policy if virtual demo or past starting point
        action = policy(state)
        mask = True
        next_state, reward, done, _ = env.step(action) # Replay transitions can not be stochastic
    else:
        action = demo.action(i)
        reward = demo.reward(i)
        mask = False
    data = (state, action, reward, next_state, done, mask)
    optimizer(data)
    
    if demo is not None and not done:
        remaining_frame_count = remaining_frame_count - 1
        done = remaining_frame_count < 0 or demo.is_lagging(score, i, L)
    i += 1
    
    if done and demo is not None:
        frame_counts_demo.append(i - max(0, demo.starting_point - K))
        success = True if score >= demo.score(len(demo) - 1) else False
    else if done:
        frame_counts_virtual.append(i)

SyntaxError: invalid syntax (<ipython-input-5-dc055d9b1aab>, line 6)

In [9]:
l = np.array([1, 2, 3])
l.mean()

2.0

In [8]:
import numpy as np

In [11]:
l.insert(1)

AttributeError: 'numpy.ndarray' object has no attribute 'insert'

In [13]:
np.append(l, 1)

array([1, 2, 3, 1])

In [15]:
np.mean([1, 2, 3])

2.0